In [19]:
%matplotlib qt

import numpy as np
from glob import glob
import os
from tqdm import tqdm
from monai.transforms import ResizeWithPadOrCrop
from pathlib import Path
import shutil
import nibabel as nib
import pickle
from monai.transforms import NormalizeIntensity
import torch
from nnunet.lib.training_utils import read_config
from nnunet.lib.utils import ConvBlocks2DGroup
from nnunet.lib.training_utils import build_2d_model
from nnunet.training.network_training.processor2 import Processor2
import matplotlib.pyplot as plt

def delete_if_exist(folder_name):
    dirpath = Path(folder_name)
    if dirpath.exists() and dirpath.is_dir():
        shutil.rmtree(dirpath)

#cropper_weights_folder_path = r"C:\Users\Portal\Documents\Isensee\nnUNet\nnunet\binary_lib"
#cropper_config = read_config(os.path.join(Path.cwd(), 'adversarial_acdc.yaml'), False, False)
#
#cropping_conv_layer = ConvBlocks2DGroup
#cropping_network = build_2d_model(cropper_config, conv_layer=cropping_conv_layer, norm=getattr(torch.nn, cropper_config['norm']), log_function=None, image_size=384, window_size=8, middle=False, num_classes=2, processor=None)
#cropping_network.load_state_dict(torch.load(os.path.join(cropper_weights_folder_path, 'model_final_checkpoint.model'))['state_dict'], strict=True)
#cropping_network.eval()
#cropping_network.do_ds = False

cropper_weights_folder_path = r"C:\Users\Portal\Documents\Isensee\nnUNet\nnunet\Quorum_cardioTrack_all_phases"
cropper_config = read_config(os.path.join(cropper_weights_folder_path, 'config.yaml'), False, False)

cropping_conv_layer = ConvBlocks2DGroup
cropping_network = build_2d_model(cropper_config, conv_layer=cropping_conv_layer, norm=getattr(torch.nn, cropper_config['norm']), log_function=None, image_size=384, window_size=8, middle=False, num_classes=4, processor=None)
cropping_network.load_state_dict(torch.load(os.path.join(cropper_weights_folder_path, 'model_final_checkpoint.model'))['state_dict'], strict=True)
cropping_network.eval()
cropping_network.do_ds = False

processor = Processor2(crop_size=192, image_size=384, cropping_network=cropping_network)

pretrained_config = read_config(os.path.join(Path.cwd(), '2d_cardiotrack_crop_normalized_2', 'config.yaml'), False, False)
pretrained_conv_layer = ConvBlocks2DGroup
pretrained_network = build_2d_model(pretrained_config, conv_layer=pretrained_conv_layer, norm=getattr(torch.nn, pretrained_config['norm']), log_function=print, image_size=192, window_size=8, middle=False, num_classes=4, processor=None)
pretrained_network.load_state_dict(torch.load(os.path.join('2d_cardiotrack_crop_normalized_2', 'model_final_checkpoint.model'))['state_dict'], strict=True)
pretrained_network.eval()
pretrained_network.do_ds = False

output_folder = 'Lib_resampling_training_mask_points'

rv_folder = os.path.join(output_folder, 'RV')
delete_if_exist(rv_folder)
Path(rv_folder).mkdir(parents=True, exist_ok=True)

lv_folder = os.path.join(output_folder, 'LV')
delete_if_exist(lv_folder)
Path(lv_folder).mkdir(parents=True, exist_ok=True)

path_list = sorted(glob(os.path.join(r"C:\Users\Portal\Documents\Isensee\nnUNet\nnunet\out\nnUNet_preprocessed\Task032_Lib\custom_experiment_planner_stage0", '*.npz')))
path_list_pkl = sorted(glob(os.path.join(r"C:\Users\Portal\Documents\Isensee\nnUNet\nnunet\out\nnUNet_preprocessed\Task032_Lib\custom_experiment_planner_stage0", '*.pkl')))
#path_list = [x for x in path_list if '_u' not in os.path.basename(x)[:-4]]

assert len(path_list) == len(path_list_pkl)
cropper = ResizeWithPadOrCrop(spatial_size=(384, 384, -1))

patient_list = sorted(list(set([os.path.basename(x).split('_')[0] for x in path_list])))

all_patient_paths = []
all_patient_paths_pkl = []
for patient in patient_list:
    patient_files = []
    patient_files_pkl = []
    for (path, pkl_path) in zip(path_list, path_list_pkl):
        if patient in path:
            patient_files.append(path)
        if patient in pkl_path:
            patient_files_pkl.append(pkl_path)
    all_patient_paths.append(sorted(patient_files))
    all_patient_paths_pkl.append(sorted(patient_files_pkl))

for (path_list, path_list_pkl) in tqdm(zip(all_patient_paths, all_patient_paths_pkl), total=len(all_patient_paths)):
    sequence = []
    filename_list = []
    pad_width_list = []
    for (npz_path, pkl_path) in zip(path_list, path_list_pkl):
        basename = os.path.basename(npz_path)
        #print(basename)

        filename = os.path.basename(npz_path)[:-4] + '.nii.gz'
        data = np.load(npz_path)
        arr = data['data']
        arr = arr.transpose((0, 2, 3, 1))

        initial_shape = list(arr.shape[1:])
        #print(arr.shape)

        pad_width = []
        for i, sp_i in enumerate([384, 384]):
            width = sp_i - initial_shape[i]
            pad_width.append((int(width // 2), int(width - (width // 2))))

        pad_vertical = 384 - initial_shape[0]
        pad_horizontal = 384 - initial_shape[1]

        cropped = cropper(arr)
        if cropped.shape[0] == 1:
            cropped = np.tile(cropped, [2, 1, 1, 1])
        sequence.append(cropped)
        filename_list.append(filename)
    sequence = np.stack(sequence, axis=0) # T, C, H, W, D
    sequence = sequence.transpose(4, 0, 1, 2, 3) # D, T, C, H, W

    sequence[:, :, 1][sequence[:, :, 1] < 0] = 0

    patient_name = basename.split('_')[0]
    point_path_list_lv = sorted(glob(os.path.join(r'custom_lib_t_4', patient_name, 'contour', 'LV', '*.npy')))
    point_path_list_rv = sorted(glob(os.path.join(r'custom_lib_t_4', patient_name, 'contour', 'RV', '*.npy')))
    assert len(point_path_list_lv) == len(point_path_list_rv) == len(sequence)

    with open(path_list_pkl[0], 'rb') as f:
        data = pickle.load(f)
        #print(data.keys())
        shape_original_after_cropping = data.get('size_after_cropping')
        size_after_resampling = data.get('size_after_resampling')
        #print(data.get('size_after_resampling'))
        #print(shape_original_after_cropping)
        rescale_y = size_after_resampling[1] / shape_original_after_cropping[1]
        rescale_x = size_after_resampling[2] / shape_original_after_cropping[2]
        rescale_factor = np.array([rescale_y, rescale_x])
        #print(rescale_y)
        #print(rescale_x)
        #print(rescale_factor.shape)
        #print(data.get('original_size_of_raw_data'))
        bbox = data.get('crop_bbox')
        #print(bbox)
        #print(pad_width)

    block = []
    padding_need_list = []
    for d in range(len(sequence)):

        lv_points = np.load(point_path_list_lv[d]) - 1
        rv_points = np.load(point_path_list_rv[d]) - 1

        # cropped data fixing
        rv_points[0] = rv_points[0] - bbox[1][0]
        rv_points[1] = rv_points[1] - bbox[2][0]

        lv_points[0] = lv_points[0] - bbox[1][0]
        lv_points[1] = lv_points[1] - bbox[2][0]
        lv_points[2] = lv_points[2] - bbox[1][0]
        lv_points[3] = lv_points[3] - bbox[2][0]

        # resampling fixing
        rv_points[0] = rv_points[0] * rescale_y
        rv_points[1] = rv_points[1] * rescale_x

        lv_points[0] = lv_points[0] * rescale_y
        lv_points[1] = lv_points[1] * rescale_x
        lv_points[2] = lv_points[2] * rescale_y
        lv_points[3] = lv_points[3] * rescale_x

        unlabeled = torch.from_numpy(sequence[d])


        #fig, ax = plt.subplots(1, 1)
        #ax.imshow(arr[0, :, :, d], cmap='gray')
        #ax.scatter(rv_points[1, :, 0], rv_points[0, :, 0])
        #ax.scatter(lv_points[1, :, 0], lv_points[0, :, 0])
        #ax.scatter(lv_points[3, :, 0], lv_points[2, :, 0])
        #while not plt.waitforbuttonpress(): pass
        #plt.close(fig)

        rv_points[0] = rv_points[0] + pad_width[0][0]
        rv_points[1] = rv_points[1] + pad_width[1][0]

        lv_points[0] = lv_points[0] + pad_width[0][0]
        lv_points[1] = lv_points[1] + pad_width[1][0]
        lv_points[2] = lv_points[2] + pad_width[0][0]
        lv_points[3] = lv_points[3] + pad_width[1][0]
        
        #fig, ax = plt.subplots(1, 1)
        #ax.imshow(unlabeled[0, 0], cmap='gray')
        #ax.scatter(rv_points[1, :, 0], rv_points[0, :, 0])
        #ax.scatter(lv_points[1, :, 0], lv_points[0, :, 0])
        #ax.scatter(lv_points[3, :, 0], lv_points[2, :, 0])
        #while not plt.waitforbuttonpress(): pass
        #plt.close(fig)

        #if unlabeled[0, 0].std() > 1000:
        #    print(unlabeled[0, 0].std())
        #    continue

        with torch.no_grad():
            mean_centroid, _ = processor.preprocess_no_registration(data=torch.clone(unlabeled[:, :1]).to('cuda:0')) # T, C(1), H, W

            #strain_mask = processor.get_strain_mask_3(torch.clone(unlabeled[:, 1]), path_list[0])
            #strain_mask = torch.from_numpy(strain_mask)
            #strain_mask = strain_mask.permute(3, 0, 1, 2)
            #unlabeled = torch.cat([unlabeled, strain_mask], dim=1)
            cropped_unlabeled, padding_need = processor.crop_and_pad(data=unlabeled, mean_centroid=mean_centroid)
            padding_need = padding_need.numpy()

            rv_points[0] = rv_points[0] - padding_need[2]
            rv_points[1] = rv_points[1] - padding_need[0]

            lv_points[0] = lv_points[0] - padding_need[2]
            lv_points[1] = lv_points[1] - padding_need[0]
            lv_points[2] = lv_points[2] - padding_need[2]
            lv_points[3] = lv_points[3] - padding_need[0]

            #fig, ax = plt.subplots(1, 1)
            #ax.imshow(cropped_unlabeled[0, 0], cmap='gray')
            #ax.scatter(rv_points[1, :, 0], rv_points[0, :, 0])
            #ax.scatter(lv_points[1, :, 0], lv_points[0, :, 0])
            #ax.scatter(lv_points[3, :, 0], lv_points[2, :, 0])
            #while not plt.waitforbuttonpress(): pass
            #plt.close(fig)

            np.save(os.path.join(lv_folder, patient_name + '_slice' + str(d+1).zfill(2) + '.npy'), lv_points)
            np.save(os.path.join(rv_folder, patient_name + '_slice' + str(d+1).zfill(2) + '.npy'), rv_points)

        #np.save(os.path.join(output_folder, current_filename[:-7] + '.npy'), out)
        #nib.save(nib.Nifti1Image(out_block, affine=np.eye(4)), os.path.join(output_folder_mask, current_filename))
        
        #with open(path_list_pkl[t], 'rb') as f:
        #    data = pickle.load(f)
        #    data['padding_need'] = padding_need
        #    data['voxelmorph_size_before'] = initial_shape
#
        #    with open(os.path.join(output_folder, os.path.basename(path_list_pkl[t])), 'wb') as f:
        #        pickle.dump(data, f)

100%|██████████| 220/220 [1:15:07<00:00, 20.49s/it]


In [ ]:
path_list = glob(os.path.join(r'custom_lib_t_4', '**', '*.gz'), recursive=True)
for path in path_list:
    print(path)
    data = nib.load(path)
    arr = data.get_fdata()
    print(arr.shape)

In [2]:
%matplotlib qt

import numpy as np
from glob import glob
import os
from tqdm import tqdm
from monai.transforms import ResizeWithPadOrCrop
from pathlib import Path
import shutil
import nibabel as nib
import pickle
from monai.transforms import NormalizeIntensity
import torch
from nnunet.lib.training_utils import read_config
from nnunet.lib.utils import ConvBlocks2DGroup
from nnunet.lib.training_utils import build_2d_model
from nnunet.training.network_training.processor2 import Processor2
import matplotlib.pyplot as plt

def delete_if_exist(folder_name):
    dirpath = Path(folder_name)
    if dirpath.exists() and dirpath.is_dir():
        shutil.rmtree(dirpath)

#cropper_weights_folder_path = r"C:\Users\Portal\Documents\Isensee\nnUNet\nnunet\binary_lib"
#cropper_config = read_config(os.path.join(Path.cwd(), 'adversarial_acdc.yaml'), False, False)
#
#cropping_conv_layer = ConvBlocks2DGroup
#cropping_network = build_2d_model(cropper_config, conv_layer=cropping_conv_layer, norm=getattr(torch.nn, cropper_config['norm']), log_function=None, image_size=384, window_size=8, middle=False, num_classes=2, processor=None)
#cropping_network.load_state_dict(torch.load(os.path.join(cropper_weights_folder_path, 'model_final_checkpoint.model'))['state_dict'], strict=True)
#cropping_network.eval()
#cropping_network.do_ds = False

cropper_weights_folder_path = r"C:\Users\Portal\Documents\Isensee\nnUNet\nnunet\Quorum_cardioTrack_all_phases"
cropper_config = read_config(os.path.join(cropper_weights_folder_path, 'config.yaml'), False, False)

cropping_conv_layer = ConvBlocks2DGroup
cropping_network = build_2d_model(cropper_config, conv_layer=cropping_conv_layer, norm=getattr(torch.nn, cropper_config['norm']), log_function=None, image_size=384, window_size=8, middle=False, num_classes=4, processor=None)
cropping_network.load_state_dict(torch.load(os.path.join(cropper_weights_folder_path, 'model_final_checkpoint.model'))['state_dict'], strict=True)
cropping_network.eval()
cropping_network.do_ds = False

processor = Processor2(crop_size=192, image_size=384, cropping_network=cropping_network)

pretrained_config = read_config(os.path.join(Path.cwd(), '2d_cardiotrack_crop_normalized_2', 'config.yaml'), False, False)
pretrained_conv_layer = ConvBlocks2DGroup
pretrained_network = build_2d_model(pretrained_config, conv_layer=pretrained_conv_layer, norm=getattr(torch.nn, pretrained_config['norm']), log_function=print, image_size=192, window_size=8, middle=False, num_classes=4, processor=None)
pretrained_network.load_state_dict(torch.load(os.path.join('2d_cardiotrack_crop_normalized_2', 'model_final_checkpoint.model'))['state_dict'], strict=True)
pretrained_network.eval()
pretrained_network.do_ds = False

cropper = ResizeWithPadOrCrop(spatial_size=(384, 384, -1))

output_folder = 'Lib_resampling_mask_points'
delete_if_exist(output_folder)

rv_folder = os.path.join(output_folder, 'RV')
Path(rv_folder).mkdir(parents=True, exist_ok=True)

lv_folder = os.path.join(output_folder, 'LV')
Path(lv_folder).mkdir(parents=True, exist_ok=True)

patient_list = glob(os.path.join(r'custom_lib_t_4', '*'))
patient_list = [x for x in patient_list if 'cfg' not in x]

for patient in tqdm(patient_list):
    print(patient)
    nifti_list = sorted(glob(os.path.join(patient, '*.gz')))
    img_list = [x for x in nifti_list if 'gt' not in x]
    gt_list = [x for x in nifti_list if 'gt' in x]
    pkl_list = sorted(glob(os.path.join(patient, '*.pkl')))

    sequence = []
    filename_list = []
    pad_width_list = []
    for img_path, gt_path in zip(img_list, gt_list):

        data_img = nib.load(img_path)
        data_gt = nib.load(gt_path)

        img = data_img.get_fdata()
        gt = data_gt.get_fdata()

        arr = np.stack([img, gt], axis=0)

        initial_shape = list(arr.shape[1:])

        pad_width = []
        for i, sp_i in enumerate([384, 384]):
            width = sp_i - initial_shape[i]
            pad_width.append((int(width // 2), int(width - (width // 2))))

        pad_vertical = 384 - initial_shape[0]
        pad_horizontal = 384 - initial_shape[1]

        cropped = cropper(arr)
        if cropped.shape[0] == 1:
            cropped = np.tile(cropped, [2, 1, 1, 1])
        sequence.append(cropped)
        filename_list.append(os.path.basename(img_path))
    sequence = np.stack(sequence, axis=0) # T, C, H, W, D
    sequence = sequence.transpose(4, 0, 1, 2, 3) # D, T, C, H, W

    sequence[:, :, 1][sequence[:, :, 1] < 0] = 0

    patient_name = os.path.basename(img_path).split('_')[0]
    point_path_list_lv = sorted(glob(os.path.join(r'custom_lib_t_4', patient_name, 'contour', 'LV', '*.npy')))
    point_path_list_rv = sorted(glob(os.path.join(r'custom_lib_t_4', patient_name, 'contour', 'RV', '*.npy')))
    assert len(point_path_list_lv) == len(point_path_list_rv) == len(sequence)
    #print(pad_width)

    block = []
    padding_need_list = []
    for d in range(len(sequence)):

        lv_points = np.load(point_path_list_lv[d]) - 1
        rv_points = np.load(point_path_list_rv[d]) - 1

        unlabeled = torch.from_numpy(sequence[d]).float()

        #fig, ax = plt.subplots(1, 1)
        #ax.imshow(arr[0, :, :, d], cmap='gray')
        #ax.scatter(rv_points[0, :, 0], rv_points[1, :, 0])
        #ax.scatter(lv_points[0, :, 0], lv_points[1, :, 0])
        #ax.scatter(lv_points[2, :, 0], lv_points[3, :, 0])
        #while not plt.waitforbuttonpress(): pass
        #plt.close(fig)

        rv_points[0] = rv_points[0] + pad_width[1][0]
        rv_points[1] = rv_points[1] + pad_width[0][0]

        lv_points[0] = lv_points[0] + pad_width[1][0]
        lv_points[1] = lv_points[1] + pad_width[0][0]
        lv_points[2] = lv_points[2] + pad_width[1][0]
        lv_points[3] = lv_points[3] + pad_width[0][0]

        #fig, ax = plt.subplots(1, 1)
        #ax.imshow(unlabeled[0, 0], cmap='gray')
        #ax.scatter(rv_points[0, :, 0], rv_points[1, :, 0])
        #ax.scatter(lv_points[0, :, 0], lv_points[1, :, 0])
        #ax.scatter(lv_points[2, :, 0], lv_points[3, :, 0])
        #while not plt.waitforbuttonpress(): pass
        #plt.close(fig)

        #if unlabeled[0, 0].std() > 1000:
        #    print(unlabeled[0, 0].std())
        #    continue

        with torch.no_grad():
            mean_centroid, _ = processor.preprocess_no_registration(data=torch.clone(unlabeled[:, :1]).to('cuda:0')) # T, C(1), H, W

            strain_mask = processor.get_strain_mask_3(torch.clone(unlabeled[:, 1]), img_list[0])
            strain_mask = torch.from_numpy(strain_mask)
            strain_mask = strain_mask.permute(3, 0, 1, 2)
            unlabeled = torch.cat([unlabeled, strain_mask], dim=1)
            cropped_unlabeled, padding_need = processor.crop_and_pad(data=unlabeled, mean_centroid=mean_centroid)
            cropped_unlabeled = cropped_unlabeled.cpu()
            padding_need = padding_need.cpu().numpy()
            padding_need_list.append(padding_need)

            rv_points[0] = rv_points[0] - padding_need[0]
            rv_points[1] = rv_points[1] - padding_need[2]

            lv_points[0] = lv_points[0] - padding_need[0]
            lv_points[1] = lv_points[1] - padding_need[2]
            lv_points[2] = lv_points[2] - padding_need[0]
            lv_points[3] = lv_points[3] - padding_need[2]

            #fig, ax = plt.subplots(1, 1)
            #ax.imshow(cropped_unlabeled[0, 0], cmap='gray')
            #ax.scatter(rv_points[0, :, 0], rv_points[1, :, 0])
            #ax.scatter(lv_points[0, :, 0], lv_points[1, :, 0])
            #ax.scatter(lv_points[2, :, 0], lv_points[3, :, 0])
            #while not plt.waitforbuttonpress(): pass
            #plt.close(fig)

            np.save(os.path.join(lv_folder, patient_name + '_slice' + str(d+1).zfill(2) + '.npy'), lv_points)
            np.save(os.path.join(rv_folder, patient_name + '_slice' + str(d+1).zfill(2) + '.npy'), rv_points)

        block.append(cropped_unlabeled)
    block = np.stack(block, axis=-1) # T, C, H, W, D
    padding_need = np.stack(padding_need_list, axis=-1) # 4, D
    
    #divisor = cropped[0].max() - cropped[0].min()
    #subtrahend = cropped[0].min()
    #cropped_img = NormalizeIntensity(subtrahend=subtrahend, divisor=divisor)(cropped[0])

    for t in range(len(block)):
        current_volume = block[t]
        current_filename = filename_list[t]

        grayscale = current_volume[0]
        grayscale_flattened = grayscale.reshape(-1, current_volume[0].shape[-1])

        if np.any((np.max(grayscale_flattened, axis=0) - np.min(grayscale_flattened, axis=0))) == 0:
            print(current_filename)
            print((np.max(grayscale_flattened, axis=0) - np.min(grayscale_flattened, axis=0)))

        grayscale_flattened = (grayscale_flattened - np.min(grayscale_flattened, axis=0)) / (np.max(grayscale_flattened, axis=0) - np.min(grayscale_flattened, axis=0) + 1e-8)
        grayscale = grayscale_flattened.reshape(current_volume[0].shape)

        #if t == 36 and '11' in path_list[0]:
        #    fig, ax = plt.subplots(1, 1)
        #    ax.imshow(grayscale[:, :, 0], cmap='gray')
        #    plt.show()
        #    plt.waitforbuttonpress()
        #    plt.close(fig)

        cropped_gt = current_volume[1]
        cropped_gt[cropped_gt < 0] = 0

        out = np.concatenate([grayscale[None], cropped_gt[None], current_volume[2:]], axis=0)
        #fig, ax = plt.subplots(1,3)
        #ax[0].imshow(current_volume[2, :, :, 1], cmap='gray')
        #ax[1].imshow(current_volume[3, :, :, 1], cmap='gray')
        #ax[2].imshow(current_volume[4, :, :, 1], cmap='gray')
        #plt.show()
        #plt.waitforbuttonpress()
        #plt.close(fig)

        np.save(os.path.join(output_folder, current_filename[:-7] + '.npy'), out)
        #nib.save(nib.Nifti1Image(out_block, affine=np.eye(4)), os.path.join(output_folder_mask, current_filename))
        
        with open(pkl_list[t], 'rb') as f:
            data = pickle.load(f)
            data['padding_need'] = padding_need
            data['voxelmorph_size_before'] = initial_shape

            with open(os.path.join(output_folder, current_filename[:-7] + '.pkl'), 'wb') as f:
                pickle.dump(data, f)

  0%|          | 0/271 [00:00<?, ?it/s]

custom_lib_t_4\patient001


  0%|          | 1/271 [00:49<3:44:10, 49.82s/it]

custom_lib_t_4\patient002


  1%|          | 2/271 [01:11<2:30:03, 33.47s/it]

...

custom_lib_t_4\patient016


  6%|▌         | 16/271 [10:22<3:00:02, 42.36s/it]

custom_lib_t_4\patient017


  6%|▌         | 16/271 [10:36<2:48:59, 39.76s/it]


KeyboardInterrupt: 